In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from skorecard import datasets
from skorecard.bucketers import DecisionTreeBucketer, OptimalBucketer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline


In [ ]:
from skorecard import datasets
from skorecard.bucketers import DecisionTreeBucketer, OptimalBucketer
from skorecard.pipeline import BucketingProcess

df = datasets.load_uci_credit_card(as_frame=True)
y = df["default"]
X = df.drop(columns=["default"])

num_cols = ["LIMIT_BAL", "BILL_AMT1"]
cat_cols = ["EDUCATION", "MARRIAGE"]

bucketing_process = BucketingProcess()#specials={'LIMIT_BAL': {'=400000.0' : [400000.0]}})
bucketing_process.register_prebucketing_pipeline(
                            DecisionTreeBucketer(variables=num_cols, max_n_bins=100, min_bin_size=0.05),
                            DecisionTreeBucketer(variables=cat_cols, max_n_bins=100, min_bin_size=0.05)
)
bucketing_process.register_bucketing_pipeline(
        OptimalBucketer(variables=num_cols, max_n_bins=10, min_bin_size=0.05),
        OptimalBucketer(variables=cat_cols,
                        variables_type='categorical',
                        max_n_bins=10,
                        min_bin_size=0.05),
)

bucketing_process.fit(X, y)

In [ ]:
bucketing_process.prebucket_table("LIMIT_BAL")

In [ ]:
bucketing_process.prebucket_table("EDUCATION")

In [ ]:
bucketing_process.prebucket_table("MARRIAGE")

In [ ]:
bucketing_process.prebucket_table("LIMIT_BAL")

In [ ]:
bucketing_process.plot_prebucket("LIMIT_BAL")

In [ ]:
bucketing_process.plot_prebucket("EDUCATION")

In [ ]:
bucketing_process.plot_bucket("LIMIT_BAL")

In [ ]:
bucketing_process.plot_bucket("EDUCATION")

In [ ]:
pipeline = make_pipeline(
    bucketing_process,
    OneHotEncoder(),
    LogisticRegression(),
)
pipeline.fit(X[num_cols], y)

In [ ]:
from skorecard.apps import BucketTweakerApp
tweaker = BucketTweakerApp(pipeline, X[num_cols], y)

# Three modes to try: ['inline','external', 'jupyterlab']
# https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e
tweaker.run_server(mode='external', port=8051)
# tweaker.stop_server()